In [722]:
##201907 이강훈 작성

## 키워드 선정
condition_list = [
#    "유상증자결정", 
#   "최대주주",
#    "투자판단",
#   "조회공시",
#    "단일판매ㆍ공급계약체결", 
#   "특허권 취득",
#    "영업실적 등에 대한 전망",
#    "수시공시",
#    "신규시설투자등",
#    "영업실적 등에 대한 전망",
#    "소송",
#    "거래정지",
#    "영업정지",
#    "기타 경영사항",
#    "관리종목",
#    "매출액",
#    "상장폐지",
#    "중단",
#    "타법인주식및출자증권처분결정",
#    "연결재무제표 기준 영업(잠정)실적",
#    "타법인주식및출자증권취득결정",
#    "우선주의 보통주",
#    "전환청구권",
#    "영업(잠정)실적(공정공시)",
#    "영업양수결정",
#    "연결재무제표기준영업(잠정)실적",
#    "주요사항보고서",
#    "최대주주변경을수반하는주식양수도계약체결",
#    "타법인 주식 및 출자증권 양수결정",
#    "주식소각결정",
#    "기술도입ㆍ이전ㆍ제휴 계약체결",
    "기타 경영사항"
    ]

In [943]:
##  1. 공시 크롤링

In [970]:
import urllib.request
import urllib.parse
import xlsxwriter
import os
import time
import sys
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
from urllib import parse

def main():
    
    dart_div_list = []
    for kk in range(len(condition_list)):
        report = parse.quote(condition_list[kk])
    
        # Default
        config_mode = 0
        config_start_year	= 2017
        config_start_month	= 1
        config_start_day	= 1
        config_end_year		= 2017
        config_end_month	= 12
        config_end_day		= 31
  ##      corp = "삼성전자"
        workbook_name = "DART_dividends.xlsx"

        # URL for Mode 0
        url_templete_0 = "http://dart.fss.or.kr/dsab002/search.ax?reportName=%s&&maxResults=100&&startDate=%s&&endDate=%s"
        # URL for Mode 1
        url_templete_1 = "http://dart.fss.or.kr/dsab002/search.ax?reportName=%s&&maxResults=100&&textCrpNm=%s"
        headers = {'Cookie':'DSAB002_MAXRESULTS=5000;'}

        #start_day = datetime(2017,11,15)
        #end_day = datetime(2017,12,15)
        start_day = datetime(config_start_year, config_start_month, config_start_day)
        end_day = datetime(config_end_year, config_end_month, config_end_day)
        delta = end_day - start_day

        for i in range(delta.days + 1):

            d = start_day + timedelta(days=i)
            rdate = d.strftime('%Y%m%d')
            print(rdate)

            if (config_mode == 0):
                handle = urllib.request.urlopen(url_templete_0 % (report,rdate, rdate))
                print("URL" + url_templete_0 % (report,rdate, rdate))
            # config mode 1
            else:
                handle = urllib.request.urlopen(url_templete_1 % (report,urllib.parse.quote(corp)))
                print("URL" + url_templete_1 % (report,corp))

            data = handle.read()
            soup = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')

            table = soup.find('table')
            trs = table.findAll('tr')
            tds = table.findAll('td')
            counts = len(tds)
            print(counts)

            #if counts > 0:
            if counts > 2:
                # Delay operation
                time.sleep(20)

                link_list = []
                docid_list = []
                date_list = []
                corp_list = []
                market_list = []
                title_list = []
                reporter_list = []
                tr_cnt = 0

                for tr in trs[1:]:
                    tr_cnt = tr_cnt + 1
                    time.sleep(1)
                    tds = tr.findAll('td')
                    link = 'http://dart.fss.or.kr' + tds[2].a['href']
                    date = tds[4].text.strip().replace('.', '-')
                    corp_name = tds[1].text.strip()
                    market = tds[1].img['title']
                    title = " ".join(tds[2].text.split())
                    reporter = tds[3].text.strip()

                    link_list.append(link)
                    date_list.append(date)
                    corp_list.append(corp_name)
                    market_list.append(market)
                    title_list.append(title)
                    reporter_list.append(reporter)
                    #print(corp_name)
                    #print(title)

                    if (market != "코넥스시장") and (market != "기타법인") and ("기재정정" not in title):
                        dart_div_sublist = []

                        print(corp_name)
                        print(title)
                        print(date)
                        handle = urllib.request.urlopen(link)
                        print(link)
                        data = handle.read()
                        soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')
                        #print(soup2)

                        test = soup2.find('a', {'href' : '#download'})['onclick']
                        words = test.split("'")
                        #print(words)
                        rcpNo = words[1]
                        dcmNo = words[3]
                        #print(rcpNo)
                        #print(dcmNo)

                        dart2 = soup2.find_all(string=re.compile('dart2.dtd'))
                        dart3 = soup2.find_all(string=re.compile('dart3.xsd'))
                        year = int(date[0:4])
                        
                        ####################################################################
                        if len(dart3) != 0:
                            link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=2&offset=4916&length=3668&dtd=dart3.xsd"
                        else:
                            link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=0&offset=0&length=0&dtd=HTML"  

                        print(link2)

                        try:
                            handle = urllib.request.urlopen(link2)
                            #print(handle)
                            data = handle.read()
                            soup3 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')

                            tables = soup3.findAll("table")

                            if len(tables) == 1 or len(tables) == 2:
                                div_table = soup3.find("table")
                                #print(div_table)
                            else:
                                print("Tables", len(tables))
                                #div_table = soup3.findAll("table")[2]
                                div_table = soup3.findAll("table")[-1]

                            
                            div_trs = div_table.findAll('tr')
                            
                            #########################################
                            contents = div_trs[0].findAll('td')[1].text.strip()
                            print(contents)
                            #########################################
                            
                        except:
                            print ("URL ERROR")
                            contents = title
                            div_cat = "URL ERROR"
                        
                        ####################################################################
                        
                        dart_div_sublist.append(date)
                        dart_div_sublist.append(corp_name)
                        dart_div_sublist.append(market)
                        dart_div_sublist.append(title)
                        dart_div_sublist.append(link)
                        dart_div_sublist.append(contents)

                        dart_div_list.append(dart_div_sublist)

        cur_dir = os.getcwd()

        # Write an Excel file

        #workbook = xlsxwriter.Workbook(workbook_name)
        #if os.path.isfile(os.path.join(cur_dir, workbook_name)):
        #	os.remove(os.path.join(cur_dir, workbook_name))
    workbook = xlsxwriter.Workbook(workbook_name)

    worksheet_result = workbook.add_worksheet('result')
    filter_format = workbook.add_format({'bold':True,
                                            'fg_color': '#D7E4BC'
                                            })

    roe_format = workbook.add_format({'bold':True,
                                          'underline': True,
                                          'num_format': '0.00%'})

    worksheet_result.set_column('A:A', 10)
    worksheet_result.set_column('B:B', 15)
    worksheet_result.set_column('C:C', 15)
    worksheet_result.set_column('D:D', 40)
    worksheet_result.set_column('E:E', 60)
    worksheet_result.set_column('F:F', 500)

    worksheet_result.write(0, 0, "날짜", filter_format)
    worksheet_result.write(0, 1, "회사명", filter_format)
    worksheet_result.write(0, 2, "분류", filter_format)
    worksheet_result.write(0, 3, "제목", filter_format)
    worksheet_result.write(0, 4, "link", filter_format)
    worksheet_result.write(0, 5, "내용", filter_format)

    for k in range(len(dart_div_list)):
        worksheet_result.write(k+1,0, dart_div_list[k][0])
        worksheet_result.write(k+1,1, dart_div_list[k][1])
        worksheet_result.write(k+1,2, dart_div_list[k][2])
        worksheet_result.write(k+1,3, dart_div_list[k][3])
        worksheet_result.write(k+1,4, dart_div_list[k][4])
        worksheet_result.write(k+1,5, dart_div_list[k][5])

    workbook.close()

# Main
if __name__ == "__main__":
	main()


20170601
URLhttp://dart.fss.or.kr/dsab002/search.ax?reportName=%EA%B8%B0%ED%83%80%20%EA%B2%BD%EC%98%81%EC%82%AC%ED%95%AD&&maxResults=100&&startDate=20170601&&endDate=20170601
42
피앤텔
기타경영사항(자율공시) (부동산 강제집행정지 조정조서 수령)
2017-06-01
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170601900655
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170601900655&dcmNo=5676118&eleId=0&offset=0&length=0&dtd=HTML
부동산 강제집행정지 조정조서 수령
삼성물산
기타경영사항(자율공시)
2017-06-01
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170601800547
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170601800547&dcmNo=5675835&eleId=0&offset=0&length=0&dtd=HTML
[정정] 단일판매ㆍ공급계약 체결
엠씨넥스
기타경영사항(자율공시) (국책과제 선정)
2017-06-01
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170601900522
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170601900522&dcmNo=5675703&eleId=0&offset=0&length=0&dtd=HTML
국책과제 선정(자율주행자동차 핵심 기술개발 사업)
화승엔터프라이즈
기타경영사항(자율공시)
2017-06-01
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170601800122
http://dart.fss.or.kr/report/viewer.do?rcpNo

KeyboardInterrupt: 

In [725]:
## exel -> csv 변환 & DB화하기

import pandas as pd

df = pd.read_excel("DART_dividends.xlsx")  # sheetname is optional
df.to_csv('DART_crawling', index=False)  # index=False prevents pandas to write row index

# oneliner
pd.read_excel("DART_dividends.xlsx").to_csv('DART_crawling', index=False)

pd_DART = pd.read_csv("DART_crawling")

In [726]:
## 종목명, 종목코드 가져오기
import requests

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다.
code_df = code_df[['회사명', '종목코드']]

# 한글로된 컬럼명을 영어로 바꿔준다.
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

In [727]:
code = []

for i in range(len(pd_DART['회사명'])):
    item_name = pd_DART.loc[i,'회사명']
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip()
    pd_DART.loc[i,'회사코드'] = code
    
    

In [728]:
pd_DART.head()

,날짜,회사명,분류,제목,link,내용,회사코드
0,2017-12-01,아이엠텍,코스닥시장,기타경영사항(자율공시) (경영권 분쟁 소송 제기),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,주주총회개최 금지 가처분 신청(사건번호2017카합5149)의 각하결정에 대한 항고,226350
1,2017-12-01,아이엠텍,코스닥시장,"기타경영사항(자율공시) (경영권 분쟁 소송 판결, 결정)",http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,주주총회개최 금지 가처분 신청(사건번호2017카합5149)\r\n 의...,226350
2,2017-12-01,프로스테믹스,코스닥시장,기타경영사항(특허권취득)(자율공시) (피부재생 또는 주름개선 효과를 가지는 인간 지...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,피부재생 또는 주름개선 효과를 가지는 인간 지방유래 줄기세포의 배양 농축액 및 이의 용도,203690
3,2017-12-01,우정바이오,코스닥시장,기타경영사항(특허권취득)(자율공시),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,멸균챔버 통과형 과산화수소 증기 멸균 처리 시스템\r\n [HYDRO...,215380
4,2017-12-01,삼성엔지니어링,유가증권시장,기타경영사항(자율공시),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,바레인 밥코 정유 프로젝트(BAPCO Modernization Program) 낙찰...,028050


In [729]:
## 한달후 종가 가져오기

from pykrx import stock
from pykrx import e3
from pykrx import bond

In [733]:
for i in range(len(pd_DART['회사코드'])):
    item_code = pd_DART.loc[i,'회사코드']
    date = pd_DART.loc[i,'날짜']
    date_from = str(pd.to_datetime(pd_DART.loc[i,'날짜'])  + pd.DateOffset(days=4))
    date = date.replace("-","")
    date_from = date_from.split(" ")
    date_from = date_from[0].replace("-","")
    
    date_after = str(pd.to_datetime(pd_DART.loc[i,'날짜'])  + pd.DateOffset(months=1))
    date_from2 = str(pd.to_datetime(pd_DART.loc[i,'날짜']) + pd.DateOffset(months=1)+ pd.DateOffset(days=4))
    date_after = date_after.split(" ")
    date_after = date_after[0].replace("-","")
    date_from2 = date_from2.split(" ")
    date_from2 = date_from2[0].replace("-","")
    
    df = stock.get_market_ohlcv_by_date(date, date_from, item_code)
    df = df.reset_index()
    print(df)
    df_stock = df.loc[0,'종가']
    pd_DART.loc[i,'종가'] = df_stock
    
    df_after = stock.get_market_ohlcv_by_date(date_after, date_from2, item_code)
    df_after = df_after.reset_index()
    print(df_after)
    df_after_stock = df_after.loc[0,'종가']
    df_after_date = df_after.loc[0,'날짜']
    pd_DART.loc[i,'한달후날짜'] = df_after_date
    pd_DART.loc[i,'한달후종가'] = df_after_stock
    

          날짜    시가    고가    저가    종가     거래량
0 2017-12-01  2710  2745  2660  2675  181799
1 2017-12-04  2735  2780  2640  2745  350417
2 2017-12-05  2765  2785  2650  2750  245960
          날짜    시가    고가    저가    종가     거래량
0 2018-01-02  2770  2920  2740  2860  287550
1 2018-01-03  2860  3030  2850  2965  509268
2 2018-01-04  2995  3045  2900  2970  454237
3 2018-01-05  3030  3030  2900  3015  318307
          날짜    시가    고가    저가    종가     거래량
0 2017-12-01  2710  2745  2660  2675  181799
1 2017-12-04  2735  2780  2640  2745  350417
2 2017-12-05  2765  2785  2650  2750  245960
          날짜    시가    고가    저가    종가     거래량
0 2018-01-02  2770  2920  2740  2860  287550
1 2018-01-03  2860  3030  2850  2965  509268
2 2018-01-04  2995  3045  2900  2970  454237
3 2018-01-05  3030  3030  2900  3015  318307
          날짜    시가     고가    저가     종가       거래량
0 2017-12-01  8520   9520  7900   8800   2117928
1 2017-12-04  8800   9750  8800   9500   1537493
2 2017-12-05  9560  12350  9000  11750  164

          날짜    시가    고가    저가    종가    거래량
0 2018-01-08  4640  4690  4470  4540  96918
1 2018-01-09  4590  4770  4560  4675  97248
2 2018-01-10  4700  4750  4600  4700  72945
          날짜    시가    고가    저가    종가    거래량
0 2017-12-07  3825  3990  3820  3860   8030
1 2017-12-08  3860  3965  3800  3945   8940
2 2017-12-11  3950  3950  3800  3855  12488
          날짜    시가    고가    저가    종가   거래량
0 2018-01-08  3820  3820  3755  3805  3321
1 2018-01-09  3810  3815  3750  3795  1863
2 2018-01-10  3735  3835  3735  3800  8588
3 2018-01-11  3835  3835  3740  3830  2415
          날짜      시가      고가      저가      종가     거래량
0 2017-12-07  112500  116000  112000  115000  108304
1 2017-12-08  113500  116000  113000  114000   72500
2 2017-12-11  112000  118500  112000  118500   53082
          날짜      시가      고가      저가      종가     거래량
0 2018-01-08  103000  104000  101500  103000   63501
1 2018-01-09  103000  106500  101000  106500  127783
2 2018-01-10  105000  107000  102500  104500   80737
3 2018-01

          날짜   시가   고가   저가   종가      거래량
0 2017-12-14  805  862  727  750  4680799
1 2017-12-15  725  749  659  717  1775217
2 2017-12-18  717  720  655  700  1342404
          날짜   시가   고가   저가   종가     거래량
0 2018-01-15  659  666  644  654  205828
1 2018-01-16  658  690  648  663  244172
2 2018-01-17  683  683  656  664  282074
3 2018-01-18  664  683  660  675  530571
          날짜    시가    고가    저가    종가    거래량
0 2017-12-14  1170  1190  1160  1175  16091
1 2017-12-15  1165  1190  1165  1170  22499
2 2017-12-18  1170  1215  1155  1170  32214
          날짜    시가    고가    저가    종가      거래량
0 2018-01-15  1095  1115  1075  1085    61582
1 2018-01-16  1100  1105  1050  1075    59830
2 2018-01-17   990   998   803   898  1121719
3 2018-01-18   900   943   899   922   123594
          날짜    시가    고가    저가    종가    거래량
0 2017-12-14  2305  2315  2305  2315   9242
1 2017-12-15  2315  2315  2310  2310  26411
2 2017-12-18  2310  2315  2310  2315   5609
          날짜    시가    고가    저가    종가    거래량
0

          날짜      시가      고가      저가      종가     거래량
0 2018-01-22  296000  297000  290500  296000   60116
1 2018-01-23  296000  302500  294000  302500  120671
2 2018-01-24  301500  305000  300500  305000   87726
          날짜    시가    고가    저가    종가    거래량
0 2017-12-20  2930  2930  2915  2915  10434
1 2017-12-21  2915  2915  2900  2900    864
2 2017-12-22  2900  2900  2895  2895  17755
          날짜    시가    고가    저가    종가    거래량
0 2018-01-22  2860  2860  2830  2845   5038
1 2018-01-23  2830  2875  2830  2870   5003
2 2018-01-24  2870  2870  2800  2820  22707
          날짜     시가     고가     저가     종가    거래량
0 2017-12-20  75800  75800  73900  74600    207
1 2017-12-21  74600  75900  74100  74600    689
2 2017-12-22  74600  77000  74000  76200  26423
          날짜     시가     고가     저가     종가    거래량
0 2018-01-22  14550  14600  14100  14200   5517
1 2018-01-23  14300  14300  13900  14200   5683
2 2018-01-24  14350  14350  13850  13900  17786
          날짜    시가    고가    저가    종가      거래량
0 2017

          날짜    시가    고가    저가    종가      거래량
0 2018-01-22  5490  5520  5280  5460   171470
1 2018-01-23  5540  5600  5360  5510   209908
2 2018-01-24  5550  5690  5420  5590   197539
3 2018-01-25  5600  5630  5490  5570   194379
4 2018-01-26  5620  6130  5580  6050  1052758
          날짜    시가    고가    저가    종가      거래량
0 2017-12-22  1555  1655  1550  1585  1682840
1 2017-12-26  1600  1710  1570  1585  4688556
          날짜    시가    고가    저가    종가      거래량
0 2018-01-22  1795  1805  1740  1740  1355739
1 2018-01-23  1600  1655  1590  1645  6752399
2 2018-01-24  1655  1665  1615  1630  2207442
3 2018-01-25  1605  1660  1580  1655  5311851
4 2018-01-26  1665  1685  1635  1685  2301685
          날짜     시가     고가     저가     종가     거래량
0 2017-12-26  15100  15500  14700  14700  232830
1 2017-12-27  14650  15200  14550  15000  164841
2 2017-12-28  15050  15350  14950  15150   85834
          날짜     시가     고가     저가     종가     거래량
0 2018-01-26  18300  18700  18050  18600  157271
1 2018-01-29  18

          날짜     시가     고가     저가     종가      거래량
0 2018-01-29  34500  34550  34100  34200   482834
1 2018-01-30  34300  34600  33800  33950   468615
2 2018-01-31  34000  34250  33400  33650   458790
3 2018-02-01  33750  33900  33300  33650   276191
4 2018-02-02  33600  34600  32850  32950  1220615
          날짜     시가     고가     저가     종가     거래량
0 2018-01-02  36000  36550  35600  36500  355321
          날짜     시가     고가     저가     종가      거래량
0 2018-01-29  34500  34550  34100  34200   482834
1 2018-01-30  34300  34600  33800  33950   468615
2 2018-01-31  34000  34250  33400  33650   458790
3 2018-02-01  33750  33900  33300  33650   276191
4 2018-02-02  33600  34600  32850  32950  1220615
          날짜     시가     고가     저가     종가    거래량
0 2018-01-02  36550  36950  36050  36450  24893
          날짜     시가     고가     저가     종가    거래량
0 2018-01-29  37450  37750  35500  35750  41131
1 2018-01-30  36200  36500  34800  34800  36927
2 2018-01-31  35300  36900  34850  36300  33412
3 2018-02-01  

In [885]:
## 긍정부정 score

pos_neg = []
  
for i in range(len(pd_DART['회사코드'])) :

  if pd_DART.loc[i,'한달후종가'] - pd_DART.loc[i,'종가'] > 0 :
    pos_neg.append("1")
  else :
    pos_neg.append("0")
   
pos_neg = pd.DataFrame(pos_neg)
pos_neg.columns = ['긍정부정']

pd_DART['긍정부정'] = pos_neg['긍정부정']
pd_DART.head()


pd_DART.to_csv('pd_DART_crawling', index=False)  # index=False prevents pandas to write row index
pd_DART = pd.read_csv('pd_DART_crawling')
pd_DART

,날짜,회사명,분류,제목,link,내용,회사코드,종가,한달후날짜,한달후종가,긍정부정
0,2017-12-01,아이엠텍,코스닥시장,기타경영사항(자율공시) (경영권 분쟁 소송 제기),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(자율공시) (경영권 분쟁 소송 제기) 주주총회개최 금지 가...,226350,2675.0,2018-01-02,2860.0,1
1,2017-12-01,아이엠텍,코스닥시장,"기타경영사항(자율공시) (경영권 분쟁 소송 판결, 결정)",http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,"기타경영사항(자율공시) (경영권 분쟁 소송 판결, 결정) 주주총회개최 ...",226350,2675.0,2018-01-02,2860.0,1
2,2017-12-01,프로스테믹스,코스닥시장,기타경영사항(특허권취득)(자율공시) (피부재생 또는 주름개선 효과를 가지는 인간 지...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(특허권취득)(자율공시) (피부재생 또는 주름개선 효과를 가지는 인간 지...,203690,8800.0,2018-01-02,7960.0,0
3,2017-12-01,우정바이오,코스닥시장,기타경영사항(특허권취득)(자율공시),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(특허권취득)(자율공시) 멸균챔버 통과형 과산화수소 증기 멸...,215380,2095.0,2018-01-02,2255.0,1
4,2017-12-01,삼성엔지니어링,유가증권시장,기타경영사항(자율공시),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(자율공시) 바레인 밥코 정유 프로젝트(BAPCO Moder...,28050,13100.0,2018-01-02,13050.0,0
5,2017-12-04,씨에스윈드,유가증권시장,기타경영사항(자율공시)(종속회사의주요경영사항),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(자율공시)(종속회사의주요경영사항) 씨에스윈드 영국,112610,23750.0,2018-01-04,28500.0,1
6,2017-12-04,태평양물산,유가증권시장,기타경영사항(자율공시),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(자율공시) 제14회 무기명식 이권부 무보증 사모 비분리형 ...,7980,4135.0,2018-01-04,3380.0,0
7,2017-12-04,오리엔트바이오,유가증권시장,기타경영사항(자율공시) (전환사채발행후 만기전 사채 취득),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(자율공시) (전환사채발행후 만기전 사채 취득) 전환사채(해...,2630,1230.0,2018-01-04,1120.0,0
8,2017-12-05,모트렉스,코스닥시장,기타경영사항(자율공시) (상환전환우선주의 보통주 전환청구),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,기타경영사항(자율공시) (상환전환우선주의 보통주 전환청구) 상환전환우선...,118990,32750.0,2018-01-05,31800.0,0
9,2017-12-05,대성엘텍,코스닥시장,기타경영사항(자율공시) (국책과제선정),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,"기타경영사항(자율공시) (국책과제선정) '주의집중, 편의성 극대화를 위...",25440,1950.0,2018-01-05,1810.0,0


In [1038]:
#pd_DART['내용'] = pd_DART['제목'] + "        "  +pd_DART['내용'] 

In [1053]:
########################################################
pd_DART = pd.read_csv('pd_DART_crawling_20170101')

In [1054]:
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
hannanum = Hannanum()
from sklearn.model_selection import train_test_split
from textblob.classifiers import NaiveBayesClassifier


# read data from file
X = pd_DART.iloc[:,5]
Y = pd_DART['긍정부정']
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.1,random_state=1) 

xtest

459                         모바일 장치를 통한 처방전 전달 방법 및 모바일 장치
950     중국 자동차 제조사인 동풍자동차그룹 계열의 동풍실업유한공사와 합자기업 설립에 관한 ...
962                        포항 특별재난지역 및 희망2018나눔캠페인 기부금 출연
126     (정품 인증수단, 정품인증 시스템)\r\n         IMPLEMENT FOR ...
678                                              감사의 중도사임
99                                     솔레노이드 분리형 N2 퍼지 노즐
49      스피노신을 포함하는 인지기능 장애 질환 예방 또는 치료용 약학조성물\r\n     ...
283                        지주회사 전환 신고에 따른 지주회사 전환 심사결과 수신
148                 동적 재구성 및 교체를 지원하는 OS 보안 커널 시스템 및 그 방법
672                             지주회사 전환신고에 대한 심사결과 통지서 접수
860     항바이러스 활성 또는 면역증진 활성을 갖는 락토바실러스 균주 판별용 조성물 및 스크...
207                                    대주주 제재에 따른 투자판단 참고
820                                       저온 염욕 부분 열처리 방법
795       차세대 염기서열분석 기술 기반의 고효율, 고해상도 조직적합성 형별 분석 방법 및 키트
659     '레보아이(Revo-i) 수술로봇시스템 (Model MSR-5000)' 식품의약품안...
528     간엽줄기세포 또는 이의 배양액을 포함하는 신경질환의 예방 또는 치료용 조성물(Com...
619                                    마이크로 칩 (Microchip)
576           

In [1055]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
pos_tagger  = Hannanum()

from sklearn.model_selection import train_test_split
from textblob.classifiers import NaiveBayesClassifier
import pandas as pd
import json
import os
from pprint import pprint

#밑의 tokenizing함수를 이해하기 편하도록 들어놓은 예시 
#tokenizing함수의 반복문 구조를 보시면 아래에 나온 결과를 반복문으로 '/'를 구분자로 결합시킵니다.
okt = Okt()
okt.pos('당신을 보고 있으니 정말로 기분이 좋아집니다') 

# 하나의 문장을 토큰화 한 후 텍스트와 품사태깅을 / 구분자로 묶어준다.
def tokenizing(docs):
   # stopwordsdic = ['기']
   # return ['/'.join(t) for t in okt.pos(docs, norm=True, stem=True)]
    return [t for t in okt.pos(docs)]
    
train_pos = [] #훈련데이터
test_pos = [] #테스트 데이터


# read data from file
X = pd_DART.iloc[:,5]
Y = pd_DART['긍정부정']
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.2,random_state=1) 

xtrain = pd.DataFrame(xtrain)
xtest = pd.DataFrame(xtest)
ytrain = pd.DataFrame(ytrain)
ytest = pd.DataFrame(ytest)

for i in range(len(xtrain['내용'])):
    try:
        train_pos0 = [tokenizing(  xtrain.loc[i,'내용']     ), ytrain.loc[i,'긍정부정']] 
        #리스트 안에 한문장에 대해서 위에서 만든 tokenizing함수를 통해서 [[토큰화텍스트],긍/부정 여부]를
        #리스트의 각문장별로 요소로 넣는다.
        train_pos.append(train_pos0)
    except:
        pass
    
for i in range(len(xtest['내용'])):
    try:
        test_pos0 = [tokenizing(  xtest.loc[i,'내용']    ), ytest.loc[i,'긍정부정']] 
        test_pos.append(test_pos0)
    except:
        pass
    
    
print(train_pos[0])



[[('대출', 'Noun'), ('채권', 'Noun'), ('의', 'Josa'), ('대손', 'Noun'), ('발생', 'Noun'), ('가능', 'Noun')], 0]


In [1056]:
#위에서 만든 데아터에서 긍/부정을 제외하고 token에 넣어준다. [[a],b] 에서 a만 넣는다고 생각하면 됨
d = [[]]
tokens = [t for d in train_pos for t in d[0]]
print(d[0])
import nltk
text = nltk.Text(tokens,name='NMSC')#nltk라이브러리를 통해서 텍스트 데이터 나열

len(set(text.tokens))#35425개의 고유 텍스트가 존재

text.vocab().most_common(10000) #vocab().most_common(10) - 텍스트 빈도 상위 10개 보여주기 즉, count_values()를 통해서 내림차순한 것과 같습니다.

[]


[(('(', 'Punctuation'), 211),
 ((')', 'Punctuation'), 209),
 (('및', 'Noun'), 186),
 (('의', 'Josa'), 165),
 (('방법', 'Noun'), 132),
 (('한', 'Josa'), 109),
 (('장치', 'Noun'), 105),
 (('\r\n         ', 'Foreign'), 93),
 (('-', 'Punctuation'), 80),
 (('을', 'Josa'), 73),
 (('전환', 'Noun'), 69),
 (('에', 'Josa'), 63),
 (('를', 'Josa'), 56),
 (('제조', 'Noun'), 54),
 (('시스템', 'Noun'), 49),
 (('관련', 'Noun'), 48),
 ((',', 'Punctuation'), 43),
 (('계약', 'Noun'), 42),
 (('이용', 'Noun'), 42),
 (('하는', 'Verb'), 42),
 (('위', 'Noun'), 42),
 (('주', 'Noun'), 40),
 (('결과', 'Noun'), 39),
 (('용', 'Noun'), 36),
 (('for', 'Alpha'), 35),
 (('사항', 'Noun'), 33),
 (('물', 'Noun'), 33),
 (('그', 'Noun'), 32),
 (('and', 'Alpha'), 31),
 (('대한', 'Noun'), 31),
 (('선정', 'Noun'), 30),
 (('사업', 'Noun'), 29),
 (('체결', 'Noun'), 29),
 (('조성', 'Noun'), 28),
 (('안내', 'Noun'), 27),
 (('이를', 'Verb'), 27),
 (('개발', 'Noun'), 27),
 (('공급', 'Noun'), 26),
 (('승인', 'Noun'), 26),
 (('제', 'Noun'), 26),
 (('청구', 'Noun'), 26),
 (('성', 'Suffix'), 

In [1057]:
#단어 빈도수가 높은 1000개의 단어만 사용하여 각 공시 문장마다의 평가지표로 삼는다.
selected_words = [f[0] for f in text.vocab().most_common(1000)]

#term_frequency()함수는 위에서 만든 selected_words의 갯수에 따라서 각 리뷰와 매칭하여 상위 텍스트가 
#각 리뷰에 얼만큼 표현되는지 빈도를 만들기 위한 함수
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_pos]
#train_x만 설명하자면 위의 결과로 도출되는 train_x의 구조는 아래와 같다
#리스트 차원으로 표현하면 [[1000개],[1000개],[1000개]....[1000개]] 가장 바깥 리스트의 갯수는 기존 train_pos의 공시갯수
test_x = [term_frequency(d) for d, _ in test_pos]
train_y = [c for _, c in train_pos] #train_pos데이터에서 각 공시별 긍/부정 여부 데이터이므로 train_pos의 공시갯수와 같은 사이즈이다.
test_y = [c for _, c in test_pos]

In [1058]:
#모델링을 하기 위해 리스트로 되어 있는 데이터 형식을 array로 바꿔주고 dtype도 실수로 바꿔준다.
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')


In [1090]:
import tensorflow as tf
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

#tensorflow.keras를 활용하여 모델의 층 입력하기
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(1000,)))#1000개를 추출했으므로 shape는1000
model.add(layers.Dense(128, activation='relu'))#1000개를 추출했으므로 shape는1000
model.add(layers.Dense(32, activation='relu'))#1000개를 추출했으므로 shape는1000
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

#모델 생성
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])
             
#모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

#예측 결과
results 

Epoch 1/10
688/688 [==============================] - 2s 3ms/step - loss: 0.6981 - binary_accuracy: 0.4782
Epoch 2/10
688/688 [==============================] - 0s 24us/step - loss: 0.6835 - binary_accuracy: 0.5770
Epoch 3/10
688/688 [==============================] - 0s 23us/step - loss: 0.6716 - binary_accuracy: 0.6206
Epoch 4/10
688/688 [==============================] - 0s 21us/step - loss: 0.6633 - binary_accuracy: 0.6526
Epoch 5/10
688/688 [==============================] - 0s 27us/step - loss: 0.6429 - binary_accuracy: 0.6788
Epoch 6/10
688/688 [==============================] - 0s 30us/step - loss: 0.6238 - binary_accuracy: 0.7209
Epoch 7/10
688/688 [==============================] - 0s 32us/step - loss: 0.6020 - binary_accuracy: 0.7558
Epoch 8/10
688/688 [==============================] - 0s 26us/step - loss: 0.5769 - binary_accuracy: 0.7878
Epoch 9/10
688/688 [==============================] - 0s 29us/step - loss: 0.5502 - binary_accuracy: 0.8009
Epoch 10/10
53/53 [==========

[0.7261067988737574, 0.5283018884793768]

In [1093]:
def predict_pos_text(text):
    token = tokenizing(text) #okt.pos로 토큰화한 단어를 정리
    tf =term_frequency(token)#토큰화된 단어를 이용해서 가장 많이 등장하는 단어와의 빈도수 체크

    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    #data는 그럼 입력받는 text가 한 문장만 받기 때문에 가장 바깥 리스트의 요소 갯수는 1이 될 것이고
    #마찬가지로 리스트 안 리스트 요소내의 갯수는 10000개가 될 것이다.
    #np.expand_dims?? 
    score = float(model.predict(data)) #새로운 데이터를 받으면 결과 예측
    if(score >= 0.5):
        print("[{}]는\n👍{:.2f}% 확률로 긍정 공시입니다.\n".format(text, score * 100))

    else:
        print("[{}]는\n🥵{:.2f}% 확률로 부정 공시입니다.\n".format(text, (1 - score) * 100))
  

In [1094]:

xtest = xtest.reset_index(drop=True)
  
xtest.index = range(len(xtest.index))

for i in range(10):
    test = xtest.loc[i,'내용']
    predict_pos_text( test )

[모바일 장치를 통한 처방전 전달 방법 및 모바일 장치]는
🥵67.41% 확률로 부정 공시입니다.

[중국 자동차 제조사인 동풍자동차그룹 계열의 동풍실업유한공사와 합자기업 설립에 관한 의향서 체결]는
👍63.91% 확률로 긍정 공시입니다.

[포항 특별재난지역 및 희망2018나눔캠페인 기부금 출연]는
🥵53.40% 확률로 부정 공시입니다.

[(정품 인증수단, 정품인증 시스템)
         IMPLEMENT FOR A GENUINE CERTIFICATION,
         A GENUINE CERTIFICATION SYSTEM]는
👍53.38% 확률로 긍정 공시입니다.

[감사의 중도사임]는
🥵55.94% 확률로 부정 공시입니다.

[솔레노이드 분리형 N2 퍼지 노즐]는
👍53.94% 확률로 긍정 공시입니다.

[스피노신을 포함하는 인지기능 장애 질환 예방 또는 치료용 약학조성물
         Pharmaceutical composition for prevention or treatment of cognitive function disorder comprising spinosin
         (중국 특허출원 제 201280068416.6호)]는
👍59.45% 확률로 긍정 공시입니다.

[지주회사 전환 신고에 따른 지주회사 전환 심사결과 수신]는
👍61.11% 확률로 긍정 공시입니다.

[동적 재구성 및 교체를 지원하는 OS 보안 커널 시스템 및 그 방법]는
🥵60.29% 확률로 부정 공시입니다.

[지주회사 전환신고에 대한 심사결과 통지서 접수]는
👍57.55% 확률로 긍정 공시입니다.



In [1095]:
predict_pos_text("현 경영진의 횡령 및 배임혐의 고소장 제출에 대한 검찰 조사 결과")

[현 경영진의 횡령 및 배임혐의 고소장 제출에 대한 검찰 조사 결과]는
🥵60.50% 확률로 부정 공시입니다.

